In [4]:
import numpy as np

In [24]:
def affine_gap_alignment(seq1, seq2, weight_match, weight_mismatch, open_gap_penalty, continue_gap_penalty):
    n, m = len(seq1), len(seq2)
    main = np.zeros((n + 1, m + 1))
    lower = np.zeros((n + 1, m + 1))
    upper = np.zeros((n + 1, m + 1))
    backtrace = np.zeros((n + 1, m + 1))
    
    backtrace[0][0] = 10e8
            
    main[0][0] = 0
    main[0][1] = -open_gap_penalty
    main[1][0] = -open_gap_penalty

    for i in range(n+1):
        upper[i][0] = n*-open_gap_penalty*10e8
        upper[0][1] = -open_gap_penalty
        backtrace[i][0] = 0
    for i in range(m+1):
        lower[0][i] = n*-open_gap_penalty*10e8
        lower[1][0] = -open_gap_penalty
        backtrace[0][i] = -1
    
    for i in range(2, n + 1):
        main[i][0] = main[i-1][0] - continue_gap_penalty
        lower[i][0] = lower[i-1][0] - continue_gap_penalty

    for i in range(2, m + 1):
        main[0][i] = main[0][i-1] - continue_gap_penalty
        upper[0][i] = upper[0][i-1] - continue_gap_penalty
        
    for i in range(n + 1):
        for j in range(m + 1):
            lower[i][j] = max(lower[i-1][j] - continue_gap_penalty, main[i-1][j] - open_gap_penalty)
            upper[i][j] = max(upper[i][j-1] - continue_gap_penalty, main[i][j-1] - open_gap_penalty)
            score_curr = main[i-1][j-1]
            score = weight_match if seq1[i-1] == seq2[j-1] else weight_mismatch
            main[i][j] = max(score_curr + score, lower[i][j], upper[i][j])
            
            # заполним вспомогательную матрицу для обратного хода
            opt = max(score_curr + score, lower[i][j], upper[i][j])
            if opt == score_curr + score:
                backtrace[i][j] = 1
            elif opt == lower[i][j]:
                backtrace[i][j] = 0
            else:
                backtrace[i][j] = -1
            
    # собираем выравнивание по матрице backtrace        
    al1 = ""
    al2 = ""
    i, j = n, m
    while i > 0 or j > 0:
        if backtrace[i][j] == 1:
            al1 = seq1[i-1] + al1
            al2 = seq2[j-1] + al2
            i -= 1
            j -= 1
        elif backtrace[i][j] == 0:
            al1 = seq1[i-1] + al1
            al2 = "_" + al2
            i -= 1
        elif backtrace[i][j] == -1:
            al1 = "_" + al1
            al2 = seq2[j-1] + al2
            j -= 1
    print(al1, al2, sep="\n")

    

In [27]:
a = "GGCTCGTTATGCATGGCCGAGCA"
b = "AGAGATTTAGCAAACCCTCTAACAGCC"

match1 = 3
mismatch1 = 2
open1 = 10
extend1 = 2

match2 = 10
mismatch2 = 2
open2 = 15
extend2 = 7

affine_gap_alignment(a, b, match1, mismatch1, open1, extend1)
print("\n")
affine_gap_alignment(a, b, match2, mismatch2, open2, extend2)

AGCAGGCTCGTTATGCATGGCCGAGCA
AGAGATTTAGCAAACCCTCTAACAGCC


GAGCAG_GCTCGTTATGCATGGCCGAGCA
AG__AGATTTAGCAAACCCTCTAACAGCC
